In [1]:
import tensorflow as tf
import numpy as np
import scipy as sc
import h5py
import time
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [2]:
MessageWordsArray = np.array([
	[ 0, 0, 0, 0],
	[ 0, 0, 0, 1],
	[ 0, 0, 1, 0],
	[ 0, 0, 1, 1],
	[ 0, 1, 0, 0],
	[ 0, 1, 0, 1],
	[ 0, 1, 1, 0],
	[ 0, 1, 1, 1],
	[ 1, 0, 0, 0],
	[ 1, 0, 0, 1],
	[ 1, 0, 1, 0],
	[ 1, 0, 1, 1],
	[ 1, 1, 0, 0],
	[ 1, 1, 0, 1],
	[ 1, 1, 1, 0],
	[ 1, 1, 1, 1]])

MessageWordsArray_test = np.array([
	[ 0, 0, 0, 0],
	[ 0, 0, 0, 1],
	[ 0, 0, 1, 0],
	[ 0, 0, 1, 1],
	[ 0, 1, 0, 0],
	[ 0, 1, 0, 1],
	[ 0, 1, 1, 0],
	[ 0, 1, 1, 1],
	[ 1, 0, 0, 0],
	[ 1, 0, 0, 1],
	[ 1, 0, 1, 0],
	[ 1, 0, 1, 1],
	[ 1, 1, 0, 0],
	[ 1, 1, 0, 1],
	[ 1, 1, 1, 0],
	[ 1, 1, 1, 1]])

In [3]:
def AddNoise(std_dev,l): #This function adds soft noise based on a value on rand values, up measures the upper limit of noise, l measures length of array that takes noise
    return(np.random.normal(0,std_dev,l))

In [4]:
def RepetitionCodes(x,n): # n is the number of times you want to repeat the code and i is the number of times you do
    temp = x
    for i in range(n-1):    
        x = np.append(x,temp)
    return(x)

In [5]:
def ber(y_true, y_pred):
    return(tf.reduce_mean(tf.cast(tf.not_equal(y_true, tf.round(y_pred)),dtype='float64')))

In [6]:
print(ber([0,0,0],[1,0,0]))

Tensor("Mean:0", shape=(), dtype=float64)


In [7]:
repitions = 8
input_vector_length = 4*repitions
hidden_layer_nodes = 4
output_vector_length = 4
learning_rate = 1e-2
training_points = input_vector_length

In [8]:
#This is to define the input and desired outputs
Input_codes = tf.placeholder(tf.float64,[None, input_vector_length],name = 'input') #This is the vector that is sent into the NN
Output_desired_codes = tf.placeholder(tf.float64,[None, output_vector_length], name = 'output_desired') #This is the desired output on the codes


In [9]:
#This is to define the transition matrices(with biases) and activations
W1 = tf.Variable(tf.random_normal([input_vector_length,output_vector_length],stddev=0.3,dtype ="float64"),name = "Input_Layer", dtype ="float64")
B = tf.Variable(tf.random_normal([1,output_vector_length],stddev=0.3,dtype = "float64"),name ="Input_Layer_biases", dtype = "float64")

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
#Output layer is defined here
Output_layer = tf.add(tf.matmul(Input_codes,W1),B)
BER = ber(Output_layer,Output_desired_codes)

In [11]:
#Cost function and optimizer
mse = tf.reduce_mean(tf.square(Output_layer - Output_desired_codes))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(mse)

In [12]:
#Setup initialization
init_op = tf.global_variables_initializer()

In [13]:
#Adding Noise to the input vectors here
coded_train = np.zeros([len(MessageWordsArray),input_vector_length])
for i in range(0,len(MessageWordsArray)):
    coded_train[i,:] = RepetitionCodes(MessageWordsArray[i,:],repitions) + AddNoise(0.05,input_vector_length)
coded_validation = np.zeros([len(MessageWordsArray),input_vector_length])
for i in range(0,len(MessageWordsArray)):
    coded_validation[i,:] = RepetitionCodes(MessageWordsArray[i,:],repitions) + AddNoise(0.05,input_vector_length)
coded_test = np.zeros([len(MessageWordsArray_test),input_vector_length])
#for i in range(0,len(MessageWordsArray_test)):
 #   coded_test[i,:] = RepetitionCodes(MessageWordsArray_test[i,:]) + AddNoise(0.6,input_vector_length)

In [19]:
epochs = 10000
batch_size = 32


with tf.Session() as sess: #Start the session
    #Intialise the variables
    writer = tf.summary.FileWriter("./logs/log_repition")
    writer.add_graph(sess.graph)
    sess.run(tf.global_variables_initializer())
    total_batch = int(training_points/batch_size) # This is used to calculate the average loss in each iteration
    #print(total_batch)
    for epoch in range(epochs):
        average_loss = 0 #initialize average_loss as zero
        for i in range(total_batch):
            #x_batch, y_batch = y_train, x_train
            _,c,w1,b,ber_final,output_layer = sess.run([optimizer,mse,W1,B,BER,Output_layer],feed_dict = {Input_codes: coded_train, Output_desired_codes: MessageWordsArray})
            average_loss = c/total_batch
            #print(w1,w2,c,_)
#             for g, v in gradients_and_vars:
#                 if g is not None:
#                     print "****************this is variable*************"
#                     print "variable's shape:", v.shape
#                     print v
#                     print "****************this is gradient*************"
#                     print "gradient's shape:", g.shape
#                     print g
            #print(w1,w2)
            #sess.run(assign_op)
            #sess.run(assign_op_w1)
            #print(iteration.value())
            #W1 = tf.add(W1,tf.multiply(tf.random_normal([input_vector_length,hidden_layer_nodes],stddev=0.1),tf.math.pow(alpha,tf.constant(i,'float32'))))
            print(sess.run(mse, feed_dict={Input_codes: coded_validation, Output_desired_codes:MessageWordsArray}))
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(average_loss),"Bit Error rate = ",ber_final)

1.4551676544631982
('Epoch:', 1, 'cost =', '1.743', 'Bit Error rate = ', 1.0)
1.203964825340037
('Epoch:', 2, 'cost =', '1.438', 'Bit Error rate = ', 1.0)
1.0060932434944125
('Epoch:', 3, 'cost =', '1.189', 'Bit Error rate = ', 1.0)
0.8543663620262696
('Epoch:', 4, 'cost =', '0.993', 'Bit Error rate = ', 1.0)
0.7414842966441186
('Epoch:', 5, 'cost =', '0.842', 'Bit Error rate = ', 1.0)
0.658578626897373
('Epoch:', 6, 'cost =', '0.730', 'Bit Error rate = ', 1.0)
0.5958746828666424
('Epoch:', 7, 'cost =', '0.648', 'Bit Error rate = ', 1.0)
0.5447362321962516
('Epoch:', 8, 'cost =', '0.586', 'Bit Error rate = ', 1.0)
0.49897010167303074
('Epoch:', 9, 'cost =', '0.535', 'Bit Error rate = ', 1.0)
0.4549319184738043
('Epoch:', 10, 'cost =', '0.491', 'Bit Error rate = ', 1.0)
0.41096082120056054
('Epoch:', 11, 'cost =', '0.448', 'Bit Error rate = ', 1.0)
0.3667078960325961
('Epoch:', 12, 'cost =', '0.405', 'Bit Error rate = ', 1.0)
0.32261067762191975
('Epoch:', 13, 'cost =', '0.362', 'Bit Er

0.008695617787255692
('Epoch:', 141, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008653429807012626
('Epoch:', 142, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008612714641228026
('Epoch:', 143, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.0085733679087455
('Epoch:', 144, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008535263051939551
('Epoch:', 145, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008498276067853654
('Epoch:', 146, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008462299456829183
('Epoch:', 147, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.00842724051430183
('Epoch:', 148, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008393008554637708
('Epoch:', 149, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008359501984127061
('Epoch:', 150, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008326605206603128
('Epoch:', 151, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.008294198756951848
('Epoch:', 152, 'cost =', '0.004', 'Bit Error rate = ', 1.0)
0.00826217783691951

0.006944620687486645
('Epoch:', 254, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006941201436655872
('Epoch:', 255, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006937842534652759
('Epoch:', 256, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006934543181600557
('Epoch:', 257, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006931302657283239
('Epoch:', 258, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006928120250000873
('Epoch:', 259, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006924995117953885
('Epoch:', 260, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006921926107909973
('Epoch:', 261, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.00691891161712261
('Epoch:', 262, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006915949580063412
('Epoch:', 263, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.006913037594098904
('Epoch:', 264, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.00691017312419838
('Epoch:', 265, 'cost =', '0.002', 'Bit Error rate = ', 1.0)
0.0069073536952657

0.00657500525693714
('Epoch:', 461, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006573520309242991
('Epoch:', 462, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006572036135247215
('Epoch:', 463, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006570552733441842
('Epoch:', 464, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006569070102272866
('Epoch:', 465, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006567588240019204
('Epoch:', 466, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006566107145026919
('Epoch:', 467, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.0065646268154718355
('Epoch:', 468, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006563147249598146
('Epoch:', 469, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006561668445483887
('Epoch:', 470, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006560190401275015
('Epoch:', 471, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006558713114947758
('Epoch:', 472, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.00655723658454

0.00626557294817799
('Epoch:', 679, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.00626420693671792
('Epoch:', 680, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006262841401165702
('Epoch:', 681, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006261476342849531
('Epoch:', 682, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006260111763035222
('Epoch:', 683, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006258747663090567
('Epoch:', 684, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.0062573840443208096
('Epoch:', 685, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006256020908132489
('Epoch:', 686, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006254658255869428
('Epoch:', 687, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.0062532960889760784
('Epoch:', 688, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006251934408833973
('Epoch:', 689, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.006250573216843387
('Epoch:', 690, 'cost =', '0.001', 'Bit Error rate = ', 1.0)
0.00624921251450

...

0.005511846676019001
('Epoch:', 9364, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.00553495400162894
('Epoch:', 9365, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005490652979680561
('Epoch:', 9366, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005524305008769711
('Epoch:', 9367, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005513327095704842
('Epoch:', 9368, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005497761533742316
('Epoch:', 9369, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005531879276884281
('Epoch:', 9370, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005498079249794795
('Epoch:', 9371, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005506727103998668
('Epoch:', 9372, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005527759016006359
('Epoch:', 9373, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005494134345226486
('Epoch:', 9374, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055163555113464866
('Epoch:', 9375, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.00

0.005506346211464634
('Epoch:', 9494, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005510747494293425
('Epoch:', 9495, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508296896079081
('Epoch:', 9496, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005507414608869584
('Epoch:', 9497, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055108916715398346
('Epoch:', 9498, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005507064739353555
('Epoch:', 9499, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055090955836867295
('Epoch:', 9500, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005509579843424947
('Epoch:', 9501, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005507085247419405
('Epoch:', 9502, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005510196919755793
('Epoch:', 9503, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055081156991767626
('Epoch:', 9504, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508060487762602
('Epoch:', 9505, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0

0.005508323234466992
('Epoch:', 9667, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.00550857330152975
('Epoch:', 9668, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.00550901416407245
('Epoch:', 9669, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055081823483483
('Epoch:', 9670, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055089419838003735
('Epoch:', 9671, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508631142381499
('Epoch:', 9672, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508374778464824
('Epoch:', 9673, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005509013944447082
('Epoch:', 9674, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508356507465396
('Epoch:', 9675, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508695731007652
('Epoch:', 9676, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508810440266283
('Epoch:', 9677, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508345134618526
('Epoch:', 9678, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.00550

0.005658340825888359
('Epoch:', 9804, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.006215417967155786
('Epoch:', 9805, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005706939554068612
('Epoch:', 9806, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0058693056161382325
('Epoch:', 9807, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005475115889222537
('Epoch:', 9808, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0054942449961633555
('Epoch:', 9809, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005855322585989727
('Epoch:', 9810, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055915658463033925
('Epoch:', 9811, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005647975562760393
('Epoch:', 9812, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005573733542861216
('Epoch:', 9813, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005440208641762614
('Epoch:', 9814, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005908794511209841
('Epoch:', 9815, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0

0.005513142211604361
('Epoch:', 9943, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055071801927788966
('Epoch:', 9944, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005507600682392351
('Epoch:', 9945, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005512052934205041
('Epoch:', 9946, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005506135972272474
('Epoch:', 9947, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005510260880137594
('Epoch:', 9948, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005509457625938077
('Epoch:', 9949, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005506479596013428
('Epoch:', 9950, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.0055115894017946365
('Epoch:', 9951, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005507460558453976
('Epoch:', 9952, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005508088991703391
('Epoch:', 9953, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.005510750120223017
('Epoch:', 9954, 'cost =', '0.000', 'Bit Error rate = ', 1.0)
0.

In [22]:
print(w1,b)

(array([[ 4.80015023e-02,  3.26034132e-01,  8.52531331e-02,
         2.12254629e-01],
       [ 1.93101725e-01,  1.62333172e-01, -2.68201350e-01,
         8.64254847e-01],
       [-1.92071152e-01,  9.03363488e-02,  1.19946481e-01,
        -1.21361555e-01],
       [-3.93414879e-02,  1.31810628e-02,  3.11261813e-01,
        -1.84658060e-01],
       [ 1.56659325e-01, -1.84946754e-01,  4.96517002e-02,
         1.88747848e-01],
       [ 1.59112127e-01,  1.08086845e-01, -7.17028850e-02,
        -3.45769213e-01],
       [-1.09722623e-01, -2.15256087e-01, -2.91299337e-01,
         1.28596004e-01],
       [-2.44758132e-01,  1.16140257e-02,  1.14479055e-01,
         3.91370766e-01],
       [ 2.05978546e-01,  1.87250221e-04,  6.85011614e-02,
        -1.92978104e-01],
       [-1.47302270e-01,  3.68441623e-01,  2.43002168e-01,
        -4.53206016e-01],
       [ 2.84078517e-01,  2.88768798e-01,  2.29850076e-01,
        -5.22575768e-02],
       [ 6.07800833e-01,  2.57088164e-01, -7.08264223e-01,
     